<a href="https://colab.research.google.com/github/Benjamin-morel/TensorFlow/blob/main/10_%5BOTP%5D_MNIST_model_pruned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---


# **ML Model: compress an image classification model**

| | |
|------|------|
| Filename | 10_[OTP]_MNIST_model_pruned.ipynb |
| Author(s) | Benjamin (contact.upside830@silomails.com) |
| Date | February 18, 2025 |
| Aim(s) | compress and quantize a model from a pruning method |
| Dataset(s) | MNIST digit dataset [[1]](https://www.kaggle.com/datasets/hojjatk/mnist-dataset) |
| Version | Python 3.10.12 - TensorFlow 2.17.1 |


<br> **!!Read before running!!** <br>
1. Fill in the inputs
2. CPU execution is enough
3. Run all and read comments

---

#### **Motivation**

Optimize a neural network model by pruning and quantifying the model. The results of these 2 optimization techniques are studied.

#### **Outline**

* Python librairies & display utilities
* Model creation and training
* Fine-tune a pre-trained model with pruning
* Quantization
* Confrontation
* References


---

## **1. Python librairies & display utilities**

In [1]:
# @title 1.1. Python librairies [RUN ME]

! pip install -q tensorflow-model-optimization

"""math"""
import numpy as np

"""file opening"""
import os
import tempfile
import zipfile

"""data"""
import pandas as pd

"""ML models"""
import tensorflow as tf

"""optimization package"""
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.python.core.keras.compat import keras

"""performances"""
from time import time
start = time()

---


## **2. Model creation and training**

### 2.1. Data acquisition

An initial model is trained using the MNIST database. This initial model has not yet been optimized.

In [2]:
def get_data_MNIST():
  mnist = tf.keras.datasets.mnist # import MNIST build-in dataset
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  return x_train, y_train, x_test, y_test

In [3]:
x_train, y_train, x_test, y_test = get_data_MNIST()
x_train, x_test = x_train / 255.0, x_test / 255.0 # normalization

### 2.2. Model architecture

In [4]:
def create_model():
  DR = 0.2 # according to [4]
  model = keras.Sequential([
      keras.layers.Flatten(input_shape=(28, 28)),
      keras.layers.Dense(128, activation='relu'),
      keras.layers.Dropout(DR),
      keras.layers.Dense(10),
      keras.layers.Softmax()
      ])

  loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=False) # loss function (outputs are not logits but proba)
  model.compile(optimizer="sgd", loss=loss_fn, metrics=['accuracy'])

  return model

### 2.3. Model training

In [5]:
def train_model(model, features, labels, **kwargs):
  kwargs.setdefault("epochs", 5)
  kwargs.setdefault("validation_split", 0)
  kwargs.setdefault("batch_size", 32)
  kwargs.setdefault("callbacks", None)
  kwargs.setdefault("verbose", 2)
  log = model.fit(features, labels, **kwargs)

  return log.history["loss"], log.history["accuracy"], log.history["val_loss"], log.history["val_accuracy"]

In [6]:
epochs = 15
validation_split = 0.2
batch_size = 64

model_MNIST = create_model()
history = train_model(model_MNIST, x_train, y_train, validation_split=validation_split, epochs=epochs, batch_size=batch_size)

Epoch 1/15
750/750 - 6s - loss: 1.0137 - accuracy: 0.7279 - val_loss: 0.5031 - val_accuracy: 0.8781 - 6s/epoch - 7ms/step
Epoch 2/15
750/750 - 5s - loss: 0.5169 - accuracy: 0.8566 - val_loss: 0.3832 - val_accuracy: 0.8995 - 5s/epoch - 7ms/step
Epoch 3/15
750/750 - 5s - loss: 0.4331 - accuracy: 0.8784 - val_loss: 0.3373 - val_accuracy: 0.9078 - 5s/epoch - 7ms/step
Epoch 4/15
750/750 - 6s - loss: 0.3899 - accuracy: 0.8887 - val_loss: 0.3107 - val_accuracy: 0.9122 - 6s/epoch - 8ms/step
Epoch 5/15
750/750 - 4s - loss: 0.3578 - accuracy: 0.8979 - val_loss: 0.2897 - val_accuracy: 0.9183 - 4s/epoch - 6ms/step
Epoch 6/15
750/750 - 5s - loss: 0.3364 - accuracy: 0.9043 - val_loss: 0.2739 - val_accuracy: 0.9227 - 5s/epoch - 7ms/step
Epoch 7/15
750/750 - 4s - loss: 0.3179 - accuracy: 0.9099 - val_loss: 0.2611 - val_accuracy: 0.9268 - 4s/epoch - 6ms/step
Epoch 8/15
750/750 - 2s - loss: 0.3031 - accuracy: 0.9142 - val_loss: 0.2501 - val_accuracy: 0.9301 - 2s/epoch - 3ms/step
Epoch 9/15
750/750 - 3s 

### 2.4. Evaluation and saving

In order to compare the initial and optimized/compressed models, model size and prediction accuracy are calculated. For this, predictions are computed with a test dataset labelled and model size is deducted after saving the model in *.keras* format.

In [7]:
def evaluate_model(model, features, labels):
  start_interference = time()
  loss, accuracy = model.evaluate(features, labels, verbose=0)
  interference_time = time()-start_interference
  print("Test loss value %0.1f \nTest accuracy: %0.1f %%" %(loss, 100*accuracy))
  return accuracy, interference_time/len(features)

In [8]:
def save_model(model, path):
  keras.models.save_model(model, path)
  print("Model correcty saved")

In [9]:
initial_accuracy, initiale_time_interf = evaluate_model(model_MNIST, x_test, y_test)

Test loss value 0.2 
Test accuracy: 94.3 %


In [10]:
path='10_MNIST_model_pruned_initial.keras'
save_model(model_MNIST, path)
initial_size = os.path.getsize(path)

Model correcty saved


---


## **3. Fine-tune pre-trained model with pruning**

The pruning process is a technique used to reduce the size and complexity of the model by removing unnecessary weights while aiming to maintain the model's performance. The importance of a weight can be determined using various criteria, such as its magnitude (magnitude-based pruning) or its impact on the loss function.

Pruning introduces sparsity into the model, meaning that many of the weights become zero. Here, the model starts its training with 50% sparsity (50% zeros in weights) and ends with 80% sparsity.

In [11]:
def create_pruned_model(initial_model, end_step):
  prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude # pruning technique = magnitude-based pruning

  pruning_params = {'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                                             final_sparsity=0.80,
                                                                             begin_step=0,
                                                                             end_step=end_step)}

  model_for_pruning = prune_low_magnitude(model_MNIST, **pruning_params)

  model_for_pruning.compile(optimizer='sgd',
                            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                            metrics=['accuracy'])

  return model_for_pruning

In [12]:
num_images = x_train.shape[0] * (1-validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

model_for_pruning = create_pruned_model(model_MNIST, end_step)

After pruning, the model is typically fine-tuned (retrained) to recover any loss in performance that may have occurred due to the removal of weights. This step helps the model to adapt to the new, sparser structure.

This callback `tfmot.sparsity.keras.UpdatePruningStep()` is responsible for updating the pruning step during training. It ensures that the pruning schedule is followed correctly, adjusting the sparsity of the model according to the defined pruning parameters.

In [13]:
epochs = 5
history_pruned = train_model(model_for_pruning, x_train, y_train, validation_split=validation_split, epochs=epochs, batch_size=batch_size, callbacks=tfmot.sparsity.keras.UpdatePruningStep())

Epoch 1/5
750/750 - 8s - loss: 0.2595 - accuracy: 0.9302 - val_loss: 0.2178 - val_accuracy: 0.9441 - 8s/epoch - 10ms/step
Epoch 2/5
750/750 - 7s - loss: 0.2588 - accuracy: 0.9305 - val_loss: 0.2156 - val_accuracy: 0.9443 - 7s/epoch - 9ms/step
Epoch 3/5
750/750 - 6s - loss: 0.2580 - accuracy: 0.9309 - val_loss: 0.2152 - val_accuracy: 0.9436 - 6s/epoch - 8ms/step
Epoch 4/5
750/750 - 4s - loss: 0.2634 - accuracy: 0.9290 - val_loss: 0.2172 - val_accuracy: 0.9417 - 4s/epoch - 5ms/step
Epoch 5/5
750/750 - 3s - loss: 0.2676 - accuracy: 0.9285 - val_loss: 0.2165 - val_accuracy: 0.9442 - 3s/epoch - 4ms/step


Such as the initial model, the pruned model is saved to deduct its size and evaluate to compute the accuracy.

In [14]:
pruned_accuracy, pruned_time_interf = evaluate_model(model_for_pruning, x_test, y_test)

Test loss value 0.2 
Test accuracy: 94.2 %


In [15]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

path='10_MNIST_model_pruned.keras'
save_model(model_for_export, path)
pruned_size = os.path.getsize(path)

Model correcty saved


---


## **4. Quantization**

Quantization is a technique used to reduce the size and computational requirements of a neural network model by reducing the precision of the weights and activations.

In [16]:
quantized_model = tfmot.quantization.keras.quantize_model(model_MNIST)

In [17]:
quantized_model.compile(optimizer='sgd',
                        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                        metrics=['accuracy'])

epochs = 1
history_quantized = train_model(quantized_model, x_train, y_train, validation_split=validation_split, epochs=epochs, batch_size=batch_size)

750/750 - 5s - loss: 0.2625 - accuracy: 0.9291 - val_loss: 0.2042 - val_accuracy: 0.9455 - 5s/epoch - 6ms/step


In [18]:
converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [19]:
def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  start_interference = time()
  for i, test_image in enumerate(x_test):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == y_test).mean()
  interference_time = time()-start_interference

  return accuracy, interference_time/len(x_test)

In [20]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

quantized_accuracy, quantized_time_interf = evaluate_model(interpreter)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.




In [21]:
path='10_MNIST_model_pruned_quantized.keras'

with open(path, 'wb') as f:
  f.write(quantized_tflite_model)
  f.close()

quantized_size = os.path.getsize(path)

---


## **5. Confrontation and conclusion**

In [22]:
def plot_comparison(size_initial, size_pruned, size_quantized, acc_initial, acc_pruned, acc_quantized, time_initial, time_pruned, time_quantized):
  df = pd.DataFrame(columns=["model", "size", "accuracy", "time for interference"])
  df.loc[0] = ['', "[bytes]", "[%]", "[ms]"]
  df.loc[1] = ['initial', "%.0f" % size_initial, "%.1f" % acc_initial, "%.3f" % time_initial]
  df.loc[2] = ['pruned', "%.0f" % size_pruned, "%.1f" % acc_pruned, "%.3f" % time_pruned]
  df.loc[3] = ['quantized', "%.0f" % size_quantized, "%.1f" % acc_quantized, "%.3f" % time_quantized]
  return print(df)

plot_comparison(initial_size, pruned_size, quantized_size, 100*initial_accuracy, 100*pruned_accuracy, 100*quantized_accuracy, 1000*initiale_time_interf, 1000*pruned_time_interf, 1000*quantized_time_interf)

       model     size accuracy time for interference
0             [bytes]      [%]                  [ms]
1    initial   840351     94.3                 0.092
2     pruned   423122     94.2                 0.076
3  quantized   105064     94.3                 0.021


Optimization of the neural network model by pruning and quantization showed very positive results in terms of model size, accuracy and inference time.

The level of accuracy remains (almost) unchanged whatever the model. However, model size is divided by 2 after pruning and by 8 after pruning and quantization.

The inference time of the quantized model was significantly reduced to 0.020 ms, around 4 times faster than the initial, pruned model. This is crucial for real-time applications where rapid inference is essential.

---


## **6. References**


| | | | | |
|------|------|------|------|------|
| Index | Title | Author(s) | Type | Comments |
|[[1]](https://www.kaggle.com/datasets/hojjatk/mnist-dataset) | MNIST digit dataset | Y. LeCun, C. Cortes, C.J.C. Burges | dataset | - |
|[[2]](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras) | Pruning in Keras example | TensorFlow | Tutoriels | - |
|[[3]](https://www.tensorflow.org/model_optimization/guide/quantization/training_example) | Quantization aware training in Keras example | TensorFlow | Tutoriels | - |

In [23]:
print("Notebook run in %.1f seconds on %s" % ((time() - start), tf.config.list_physical_devices(device_type=None)[-1][-1]))

Notebook run in 116.6 seconds on CPU
